# **Anchors**

In [4]:
pip install anchor-exp


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
import pandas as pd
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Define the datasets**

In [11]:
training_folder = '../datasets/dataset5000.csv'
validation_folder = '../datasets/dataset500.csv'

training_df_unsplit = pd.read_csv(training_folder)
validation500_df = pd.read_csv(validation_folder)

#Split 80 20 the training dataset in training anda validation to have more similar data
train_percentage = 80
val_percentage = 20
indices = np.arange(0,5000)
np.random.seed(1234)
indices = np.random.permutation(indices)
training_indices = indices[0:int(5000*train_percentage/100)]
validation_indices = indices[int(5000*train_percentage/100):]

training_df = training_df_unsplit.iloc[training_indices]
validation_df = training_df_unsplit.iloc[validation_indices]
print('Training dataset size: ', training_df.shape)
print('Validation dataset size: ', validation_df.shape)

#create a csv with the new training data and save it
training_df.to_csv('../datasets/training_dataset.csv', index=False)
validation_df.to_csv('../datasets/validation_dataset.csv', index=False)

# take only the values that are true for all the requirements and drop only the requirements columns
all_true_training_dataset = training_df[(training_df['req_0'] == 1) & (training_df['req_1'] == 1) & (training_df['req_2'] == 1) & (training_df['req_3'] == 1)]
all_true_training_dataset = all_true_training_dataset.drop(columns=['req_0','req_1', 'req_2', 'req_3'])

req_names = ['req_0', 'req_1', 'req_2', 'req_3']
feature_names = ['cruise speed','image resolution','illuminance','controls responsiveness','power','smoke intensity','obstacle size','obstacle distance','firm obstacle']

Training dataset size:  (4000, 13)
Validation dataset size:  (1000, 13)


In [12]:
training_folder = '../datasets/training_dataset.csv'
validation_folder = '../datasets/validation_dataset.csv'

# Upsampling

In [13]:
# All the samples with req_0..3 = 1
all_true = training_df[
    (training_df['req_0'] == 1) &
    (training_df['req_1'] == 1) &
    (training_df['req_2'] == 1) &
    (training_df['req_3'] == 1)
]

# Just the samples with req_1 = 1(can include some that don't have all the others at 1)
#req1_true = training_df[training_df['req_1'] == 1]

# Upsample: repeat the same rows multiple times
all_true_upsampled = pd.concat([all_true] * 5, ignore_index=True)   # multiplied x5
#req1_upsampled = pd.concat([req1_true] * 3, ignore_index=True)      # multiplied x3

# Exclude rows already included in all_true or req1_true (without duplicates)
#indices_to_remove = all_true.index.union(req1_true.index)
training_df_no_dupes = training_df.drop(index=all_true.index)

# Create the new balanced dataset
training_df_balanced = pd.concat(
    [training_df_no_dupes, all_true_upsampled],
    ignore_index=True
)

# Shuffle ad save the new dataset
training_df_balanced = training_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
training_df_balanced.to_csv('../datasets/training_balanced.csv', index=False)

print('Final upsampled training set size:', training_df_balanced.shape)

training_folder = '../datasets/training_balanced.csv' #comment this line if you do not want to use the upsampled dataset 

Final upsampled training set size: (4624, 13)


# Train the model and the anchors

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

models = [] #will contain the models (one per requirement)
datasets = [] #will contain the datasets as needed by the anchor library

explanations = np.zeros((len(req_names), all_true_training_dataset.shape[0]), dtype=object) #will contain the explanations (objects)
exp_txt = [] #will contain the textual explanations its structure is a matrix (list of lists) where each row corresponds to a requirement 
             #and each column corresponds to the explanation for the corresponding row in all_true_training_dataset

feature_to_use = [i for i in range(9)] #contains the range of features to use

for i in range(len(req_names)):
    
    #we load the dataset in anchors
    datasets.append(\
        utils.load_csv_dataset(\
            training_folder, 9+i,\
            features_to_use=feature_to_use,\
            categorical_features=None))

    models.append(\
            sklearn.ensemble.GradientBoostingClassifier(random_state=1234))
    models[i].fit(datasets[i].train, datasets[i].labels_train)
    
    #initialize the explainer
    explainer = anchor_tabular.AnchorTabularExplainer(
        datasets[i].class_names, #it maps the 0 and 1 in the dataset's requirements to the class names
        datasets[i].feature_names,
        datasets[i].train,
        datasets[i].categorical_names)
    
    #explain only points satisfying all the requirements
    names = []
    
    for j in range(all_true_training_dataset.shape[0]):
        exp = explainer.explain_instance(all_true_training_dataset.iloc[j].values.reshape(1, -1), models[i].predict, threshold=0.95) #0.95
        explanations[i,j] = exp
        names.append(exp.names())        
    
    exp_txt.append(names)
    
    print(exp_txt[i])


[["b'firm obstacle' = 1.0", "b'smoke intensity' <= 21.27", "b'illuminance' > 53.75", "b'image resolution' > 28.13", "b'cruise speed' <= 72.78", "b'obstacle size' > 27.47"], ["b'cruise speed' <= 24.24", "b'smoke intensity' <= 21.27", "b'illuminance' > 28.85", "b'image resolution' > 52.73", "b'power' > 52.00", "b'obstacle size' > 27.47"], ["b'firm obstacle' = 1.0", "53.75 < b'illuminance' <= 77.43", "b'smoke intensity' <= 46.38", "b'image resolution' > 28.13", "b'obstacle size' > 50.25", "26.00 < b'power' <= 52.00", "b'controls responsiveness' > 29.40"], ["b'firm obstacle' = 1.0", "b'illuminance' > 77.43", "b'smoke intensity' <= 21.27", "46.95 < b'cruise speed' <= 72.78", "b'power' > 52.00", "b'obstacle distance' <= 25.33", "b'controls responsiveness' > 29.40"], ["b'firm obstacle' = 1.0", "b'image resolution' > 52.73", "b'illuminance' > 53.75", "b'cruise speed' <= 72.78", "b'smoke intensity' <= 46.38", "b'obstacle distance' <= 74.84"], ["b'firm obstacle' = 1.0", "b'image resolution' > 77

# **!!!Notice this:**

In [15]:
datasets[0].train.shape

(3699, 9)

We are only using the 92.5% of our training dataset... may it be the reason for anchors not working properly?

In [ ]:
for i in range(len(req_names)):
    print(f"Model {i+1} training accuracy: {accuracy_score(datasets[i].labels_train, models[i].predict(datasets[i].train)):.4f}")

Model 1 training accuracy: 0.9470
Model 2 training accuracy: 0.9002
Model 3 training accuracy: 0.9476
Model 4 training accuracy: 0.9389


**Let us make some checks...**

In [ ]:
np.count_nonzero(explanations[0] == explanations[1]) #count the number of times the explanations are the same for the two requirements

0

In [ ]:
len(exp_txt), len(exp_txt[0])

(4, 156)

**Define useful data-wrangling functions**

function separating the name of the feature from the ranges

In [ ]:
def get_anchor(a):
    quoted_part = a.split("'")[1]
    rest = a.replace(f"'{quoted_part}'", '').replace("b", '').strip()

    return quoted_part, rest

function creating the intervals

In [ ]:
import re
from math import inf

def parse_range(expr: str):
    expr = expr.strip().replace(" ", "")
    
    patterns = [
        (r"^=(\-?\d+(\.\d+)?)$", 'equals'),
        (r"^(>=|>)\s*(-?\d+(\.\d+)?)$", 'lower'),
        (r"^(<=|<)\s*(-?\d+(\.\d+)?)$", 'upper'),
        (r"^(-?\d+(\.\d+)?)(<=|<){1,2}(<=|<)(-?\d+(\.\d+)?)$", 'between'),
        (r"^(-?\d+(\.\d+)?)(>=|>){1,2}(>=|>)(-?\d+(\.\d+)?)$", 'reverse_between'),
    ]
    
    for pattern, kind in patterns:
        match = re.match(pattern, expr)
        if match:
            if kind == 'equals':
                num = float(match.group(1))
                return (num, num, True, True)
            elif kind == 'lower':
                op, num = match.group(1), float(match.group(2))
                return (
                    num,
                    inf,
                    op == '>=',
                    False
                )
            elif kind == 'upper':
                op, num = match.group(1), float(match.group(2))
                return (
                    -inf,
                    num,
                    False,
                    op == '<='
                )
            elif kind == 'between':
                low = float(match.group(1))
                op1 = match.group(3)
                op2 = match.group(4)
                high = float(match.group(5))
                return (
                    low,
                    high,
                    op1 == '<=',
                    op2 == '<='
                )
            elif kind == 'reverse_between':
                high = float(match.group(1))
                op1 = match.group(3)
                op2 = match.group(4)
                low = float(match.group(5))
                return (
                    low,
                    high,
                    op2 == '>=',
                    op1 == '>='
                )

    raise ValueError(f"Unrecognized format: {expr}")

function that returns the interseption among two intervals 

In [ ]:
from typing import Optional, Tuple

def intersect(
    a: Tuple[float, float, bool, bool],
    b: Tuple[float, float, bool, bool]
) -> Optional[Tuple[float, float, bool, bool]]:
    
    a_low, a_high, a_li, a_ui = a
    b_low, b_high, b_li, b_ui = b

    # Compute max of lower bounds
    if a_low > b_low:
        low, li = a_low, a_li
    elif a_low < b_low:
        low, li = b_low, b_li
    else:
        low = a_low
        li = a_li and b_li

    # Compute min of upper bounds
    if a_high < b_high:
        high, ui = a_high, a_ui
    elif a_high > b_high:
        high, ui = b_high, b_ui
    else:
        high = a_high
        ui = a_ui and b_ui

    # Check for empty intersection
    if low > high:
        return None
    if low == high and not (li and ui):
        return None

    return (low, high, li, ui)

function that return the truth value of a num (val) being inside a given interval

In [ ]:
def inside(val, interval):
    low, high, li, ui = interval
    if li and ui:
        return low <= val <= high
    elif li and not ui:
        return low <= val < high
    elif not li and ui:
        return low < val <= high
    else:
        return low < val < high

**Wrangle the data to cope better with them**

Transform exp_txt in exp_dict a list of 4 dictionaries per element (one per requirement) in which are listed each feature with the respective constraints (as a range data structure)
The range data structure is a 4-element tuple (float, float, boolean, boolean) where (a,b,x,y) num $\in$ (a,b) and x and y are true if the extremes are included, otherwise they are false

In [ ]:
exp_dict =[]
for i in range(len(exp_txt[0])):
    exp_dict.append([{}, {}, {}, {}])
    for j in range(len(req_names)):
        for k in range(len(exp_txt[j][i])):
            quoted, rest = get_anchor(exp_txt[j][i][k])
            exp_dict[i][j][quoted] = parse_range(rest)

**Intersect all the obtained explanations over the different requirements**

In [ ]:
exp_intersected = []

for i in range(len(exp_dict)):
    exp_intersected.append(exp_dict[i][3])
    for j in range(len(req_names)-1):
        for k in feature_names:
            if k in exp_dict[i][j]:
                if k in exp_intersected[i]:
                    inter = intersect(exp_dict[i][j][k], exp_intersected[i][k])
                    if inter is not None:
                        exp_intersected[i][k] = inter
                    else:
                        raise ValueError(f"Unrecognized format: {exp_dict[i][j][k]} and {exp_dict[i][j+1][k]}")
                else:
                    exp_intersected[i][k] = exp_dict[i][j][k]

# Validation

Define the function classifing w.r.t. the anchors

In [ ]:
def classify_w_anchor(input, thresholds):
    inside_points = []
    pos = []

    featureNames = ['cruise speed','image resolution','illuminance','controls responsiveness','power',
     'smoke intensity','obstacle size','obstacle distance','firm obstacle']
    
    for i in range(input.shape[0]):
        for j in range(len(thresholds)):
            flag = True
            for k in featureNames:
                if k in thresholds[j]:
                    if not (inside(input.iloc[i][k], thresholds[j][k])):
                        flag = False
                        break
            if flag:
                inside_points.append(input.iloc[i])
                pos.append(i)
                break
            else:
                flag = True
    return inside_points, pos

Verify if the function works properly by submitting the whole 5k dataset, we should be obtaining only the previously obtained samples for which all the req. are satisfied

In [ ]:
sat = []
for i in range(all_true_training_dataset.shape[0]):
    for j in range(len(exp_intersected)):
        flag = True
        for k in feature_names:
            if k not in exp_intersected[j]:
                continue
            else:
                if not (inside(all_true_training_dataset.iloc[i][k], exp_intersected[j][k])):
                    print(all_true_training_dataset.iloc[i])
                    flag = False
                    break
        
        if flag:
            sat.append(all_true_training_dataset.iloc[i])
            break
        else:
            flag = True

len(sat), len(all_true_training_dataset)

cruise speed               15.0502
image resolution           75.2163
illuminance                80.4418
controls responsiveness    59.9659
power                      91.0000
smoke intensity             6.5466
obstacle size              42.9558
obstacle distance          41.2781
firm obstacle               0.0000
Name: 3123, dtype: float64
cruise speed               74.4545
image resolution           42.4752
illuminance                57.3480
controls responsiveness    89.8644
power                      44.0000
smoke intensity            34.5672
obstacle size              92.9541
obstacle distance          86.9924
firm obstacle               1.0000
Name: 1862, dtype: float64
cruise speed               74.4545
image resolution           42.4752
illuminance                57.3480
controls responsiveness    89.8644
power                      44.0000
smoke intensity            34.5672
obstacle size              92.9541
obstacle distance          86.9924
firm obstacle               1.0000
N

(155, 156)

Now we will see how many points belongng to the dataset with values that correspond to all true values for all requirements gets predicted by the model we are using to train the anchors.

In [ ]:
targets = np.zeros((all_true_training_dataset.shape[0], len(models)))
for i in range(all_true_training_dataset.shape[0]):
    for j in range(len(models)):
        targets[i,j] = models[j].predict(all_true_training_dataset.iloc[i].values.reshape(1, -1))

#print(targets)
all_yes_t = np.all(targets==1.0, axis = 1)

correct_per_model = np.sum(targets == 1, axis=0)
print(f"Correct 'True' predictions per model on all-true samples: {correct_per_model}")

true_mask = targets[:, 1] == 1
model2_preds = models[1].predict(all_true_training_dataset.values)  
true_positives = np.sum((model2_preds == 1) & true_mask)

print(f"Model 2 true positive rate: {true_positives} / {np.sum(true_mask)}")


Correct 'True' predictions per model on all-true samples: [156 142 148 153]
Model 2 true positive rate: 142 / 142


In [ ]:
all_yes_t.shape

(156,)

In [ ]:
np.count_nonzero(all_yes_t==1), targets.shape, len(sat)

(134, (156, 4), 155)

Validate the function on a validation set (20%  of the 5k dataset)

In [ ]:
n_samples_val = validation_df.shape[0]
print(f"Number of samples in validation set: {n_samples_val}")

Number of samples in validation set: 1000


In [ ]:
y_pred = np.zeros((n_samples_val, len(models)))
y_real = np.zeros((n_samples_val, len(models)))


for i in range(n_samples_val):

    y_real[i] = validation_df.iloc[i].values[9:] #These are the real labels the validation set has
    #print(y_real[i])

    values = validation_df.iloc[i].values[:9]
    for j in range(len(models)):
        y_pred[i,j] = models[j].predict(values.reshape(1, -1))
        

#accuracies of the models
for i in range(len(models)):
    print(f"Model {i+1} validation accuracy: {accuracy_score(y_real[:,i], y_pred[:,i])}")
    


Model 1 validation accuracy: 0.898
Model 2 validation accuracy: 0.863
Model 3 validation accuracy: 0.915
Model 4 validation accuracy: 0.901


In [ ]:
print(y_real.shape)
all_true_testing_dataset = validation_df[(validation_df['req_0'] == 1) & (validation_df['req_1'] == 1) & (validation_df['req_2'] == 1) & (validation_df['req_3'] == 1)]
print(all_true_testing_dataset.shape)
#print(all_true_testing_dataset)

all_true_yreal = y_real[(validation_df['req_0'] == 1) & (validation_df['req_1'] == 1) & (validation_df['req_2'] == 1) & (validation_df['req_3'] == 1)]
print(all_true_yreal.shape)
print(all_true_yreal)

(1000, 4)
(49, 13)
(49, 4)
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [ ]:
y_anch = np.zeros(n_samples_val)
for i in range(n_samples_val):
    y_anch[i] = 0

    for j in range(len(exp_intersected)):
        flag = True
        for k in feature_names:
            if k in exp_intersected[j]:
                if not (inside(validation_df.iloc[i][k], exp_intersected[j][k])):
                    flag = False
                    break
        if flag:
            y_anch[i] = 1
            break 

In [ ]:
y_real_all = y_real[(validation_df['req_0'] == 1) & (validation_df['req_1'] == 1) & (validation_df['req_2'] == 1) & (validation_df['req_3'] == 1)]
print(all_true_yreal.shape)
print(all_true_yreal)

y_pred_all = np.all(y_pred==1, axis = 1)
y_anch_all = y_anch

(49, 4)
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [ ]:
y_real_all[y_real_all == True].shape, y_pred_all[y_pred_all == True].shape, y_anch_all[y_anch_all == True].shape

((196,), (75,), (53,))

In [ ]:
y_real_all = np.zeros(n_samples_val)
y_real_all = np.all(y_real==1, axis = 1)

In [ ]:
y_real_all.shape, np.count_nonzero(y_real_all==1), np.count_nonzero(y_pred_all==1), np.count_nonzero(y_anch_all==1)

((1000,), 49, 75, 53)

In [ ]:
count_r_a = 0
real_ancors = []
count_a_p = 0
anchors_pred = []
count_r_p = 0
real_pred = []
for i in range(n_samples_val):
    if(y_real_all[i]==1 and y_anch_all[i]==1):
        count_r_a += 1
        real_ancors.append(i)
    if(y_anch[i]==1 and y_pred_all[i]==1):
        count_a_p += 1
        anchors_pred.append(i)
    if(y_real_all[i]==1 and y_pred_all[i]==1):
        count_r_p +=1
        real_pred.append(i)


print(f"Number of samples in validation set with all true requirements and anchor: {count_r_a}")
print(f"Number of samples in validation set with all true requirements and anchor predicted as true by the model: {count_a_p}")
print(f"Num samples predicted as true by the model and with all true requirements: {count_r_p}")


Number of samples in validation set with all true requirements and anchor: 18
Number of samples in validation set with all true requirements and anchor predicted as true by the model: 37
Num samples predicted as true by the model and with all true requirements: 27


In [ ]:
# Convert to sets for clean intersection operations
real_anchors = set(real_ancors)       # note spelling fix
anchors_pred = set(anchors_pred)
real_pred = set(real_pred)

# Compute intersections
common_r_a_a_p = len(real_anchors & anchors_pred)
common_r_a_r_p = len(real_anchors & real_pred)
common_a_p_r_p = len(anchors_pred & real_pred)

# Print results
print("Number of samples common to the real&anchors and the anchors&predicted: ", common_r_a_a_p)
print("Number of samples common to the real&anchors and the predicted&real: ", common_r_a_r_p)
print("Number of samples common to the anchors&predicted and the predicted&real: ", common_a_p_r_p)

common_all = len(real_anchors & anchors_pred & real_pred)
print("Number of samples common to real, anchor, and predicted: ", common_all)


Number of samples common to the real&anchors and the anchors&predicted:  17
Number of samples common to the real&anchors and the predicted&real:  17
Number of samples common to the anchors&predicted and the predicted&real:  17
Number of samples common to real, anchor, and predicted:  17


In [ ]:
missed_by_anchor = []

for i in range(n_samples_val):
    if np.all(y_real[i] == 1) and y_anch[i] == 0:
        missed_by_anchor.append(i)

print("Missed by anchor despite real reqs being true:", missed_by_anchor)
print(f"Number of samples missed by anchor: {len(missed_by_anchor)}")

for idx in missed_by_anchor[:1]:  # just check the first
    print(f"Sample {idx} values:")
    print(validation_df.iloc[idx][:9])  # show features
    print("Requirements:", y_real[idx])

Missed by anchor despite real reqs being true: [68, 69, 75, 158, 159, 172, 217, 262, 277, 292, 298, 392, 474, 485, 500, 504, 537, 554, 558, 568, 588, 685, 689, 712, 756, 765, 777, 801, 865, 896, 934]
Number of samples missed by anchor: 31
Sample 68 values:
cruise speed                 3.287
image resolution           31.8627
illuminance                46.4804
controls responsiveness    70.0194
power                         67.0
smoke intensity            84.6223
obstacle size              64.6357
obstacle distance          61.8915
firm obstacle                  1.0
Name: 3089, dtype: object
Requirements: [1. 1. 1. 1.]


In [ ]:
false_positives_anchor = []

for i in range(n_samples_val):
    if y_anch[i] == 1 and not np.all(y_real[i] == 1):
        false_positives_anchor.append(i)

print("Anchor predicted 1 but real requirements not all true:", false_positives_anchor)
print(f"Number of samples where anchor predicted 1 but real requirements not all true: {len(false_positives_anchor)}")

false_positives_anchor_req0_count = 0
false_positives_anchor_req1_count = 0
false_positives_anchor_req2_count = 0
false_positives_anchor_req3_count = 0
for i in range(len(false_positives_anchor)):
    if validation_df.iloc[false_positives_anchor[i]]['req_0'] == 0:
        false_positives_anchor_req0_count += 1
    if validation_df.iloc[false_positives_anchor[i]]['req_1'] == 0:
        false_positives_anchor_req1_count += 1
    if validation_df.iloc[false_positives_anchor[i]]['req_2'] == 0:
        false_positives_anchor_req2_count += 1
    if validation_df.iloc[false_positives_anchor[i]]['req_3'] == 0:
        false_positives_anchor_req3_count += 1

print("False positives anchor req 0 count: ", false_positives_anchor_req0_count)
print("False positives anchor req 1 count: ", false_positives_anchor_req1_count)
print("False positives anchor req 2 count: ", false_positives_anchor_req2_count)
print("False positives anchor req 3 count: ", false_positives_anchor_req3_count)
    

Anchor predicted 1 but real requirements not all true: [12, 66, 83, 90, 111, 143, 203, 233, 265, 318, 373, 387, 396, 428, 436, 462, 468, 502, 514, 534, 634, 646, 654, 688, 750, 779, 780, 817, 841, 843, 846, 877, 919, 937, 957]
Number of samples where anchor predicted 1 but real requirements not all true: 35
False positives anchor req 0 count:  6
False positives anchor req 1 count:  33
False positives anchor req 2 count:  9
False positives anchor req 3 count:  8


In [ ]:
r_p = np.zeros(n_samples_val)
r_a = np.zeros(n_samples_val)
p_a = np.zeros(n_samples_val)

for i, req in enumerate(req_names):
    for j in range(n_samples_val):
        r_p[j] = y_pred[j,i] or y_real[j,i]
        r_a[j] = y_anch[j] or y_real[j,i]
        p_a[j] = y_anch[j] or y_pred[j,i]

r_p = np.sum(r_p)
r_a = np.sum(r_a)
p_a = np.sum(p_a)
r_p, r_a, p_a, [np.count_nonzero(y_real[:,i]) for i in range(4)]


(293.0, 269.0, 234.0, [342, 172, 235, 261])